In [64]:
import json

with open("../saq_cat.json") as f:
    data = json.load(f)


# print number of entries where upc  is NULL
count = 0
for entry in data:
    if entry['upc'] == None:
        print(entry)
        count += 1
    
print(count)

{'name': 'Smirnoff Ice', 'appellation': None, 'format': '24 x 330 ml', 'country': 'États-Unis', 'region': None, 'price': '61,20\xa0$', 'alcohol': '7 %', 'sugar': '>60 g/L', 'color': None, 'producer': 'The Smirnoff Co.', 'agent': 'Diageo Canada Inc.', 'code': '10754711', 'upc': None, 'tasting': None, 'aging': None, 'temperature': None, 'grapes': None, 'misc': None, 'continent': 'Amérique du Nord', 'category': 'CATEGORY_SPIRITS', 'alcoholType': 'liquor'}
{'name': 'Louis Roederer Cristal Brut 1999', 'appellation': 'Champagne', 'format': '3 L', 'country': 'France', 'region': 'Champagne, Montagne de Reims', 'price': '3\xa0295,00\xa0$', 'alcohol': '12 %', 'sugar': None, 'color': 'Blanc', 'producer': 'Louis Roederer', 'agent': 'Authentic Vins et Spiritueux Inc', 'code': '10836119', 'upc': None, 'tasting': None, 'aging': None, 'temperature': None, 'grapes': 'Pinot noir\xa055\xa0%, Chardonnay\xa045\xa0%', 'misc': None, 'continent': 'Europe', 'millesime': 1999, 'category': 'CATEGORY_WINE', 'alco

In [65]:
# Extract unique categories and alcoholTypes
categories = {item["category"] for item in data if "category" in item}
alcohol_types = {item["alcoholType"] for item in data if "alcoholType" in item}
appellations = {item["appellation"] for item in data if "appellation" in item}

#replace empty strings with "None"
categories = {item if item != "" else "None" for item in categories}
alcohol_types = {item if item != "" else "None" for item in alcohol_types}
appellations = {item if item != "" else "None" for item in appellations}

# Filter out None values and convert to sorted lists
categories = sorted(item for item in categories if item is not None)
alcohol_types = sorted(item for item in alcohol_types if item is not None)
appellations = sorted(item for item in appellations if item is not None)

print("Distinct Categories:", categories)
print("Distinct Alcohol Types:", alcohol_types)
print("Distinct Appellations:", appellations)

Distinct Categories: ['CATEGORY_BEER', 'CATEGORY_SPIRITS', 'CATEGORY_WINE']
Distinct Alcohol Types: ['beer', 'liquor', 'wine']
Distinct Appellations: ['Abruzzo', 'Achel', 'Aglianico del Taburno/Taburno', 'Aglianico del Vulture', 'Alella', 'Alenquer', 'Alentejo', 'Alexander Valley', 'Alicante', 'Almansa', 'Aloxe-Corton', 'Aloxe-Corton Premier Cru', 'Alsace', 'Alsace Côte de Rouffach', 'Alsace Grand Cru', 'Alta Langa', 'Alto Adige/Südtirol', 'Amarone della Valpolicella', 'Ancient Lakes of Columbia Valley', 'Anderson Valley', 'Anjou', 'Arbois', 'Arbois Pupillin', 'Armagnac', 'Arroyo Seco', 'Asolo Prosecco', 'Asti', 'Auxey-Duresses', 'Auxey-Duresses Premier Cru', 'Baden', 'Bairrada', 'Balaton', 'Balatonboglár', 'Bandol', 'Banyuls', 'Barbaresco', "Barbera d'Alba", "Barbera d'Asti", 'Barco Reale di Carmignano', 'Bardolino', 'Barolo', 'Barsac', 'Bas Armagnac', 'Beamsville Bench', 'Beaujolais', 'Beaujolais Villages', 'Beaumes de Venise', 'Beaune', 'Beaune Premier Cru', 'Beira Interior', "Beni-

In [53]:
import psycopg2

# Connect to the default "postgres" database to create a new one
conn = psycopg2.connect(
    host="34.47.32.127",
    port=5432,  # Default PostgreSQL port
    dbname="octogone_cat_v0",
    user="postgres",
    password="f87JpR9Uvud6NR3HwbP"
)


# INSERT Wines

In [55]:
import json
import pandas
from uuid import uuid4
import numpy as np
with open("../saq_cat.json") as f:
    data = json.load(f)

# put data into a dataframe
df = pandas.DataFrame(data)
df.head()
df_wine = df[df["category"] == "CATEGORY_WINE"].copy()
print(f"Number of wine entries: {len(df_wine)}")
df_wine.head()
print(df_wine.columns)


df_wine.loc[:, 'appellation'] = df_wine['appellation'].fillna('Other')

#in application_df replade the string '(e)' with ''
df_wine.loc[:, 'color'] = df_wine['color'].str.replace(r'\(e\)', '', regex=True)  # Explicit regex

# replace all NaN values with None in each column of df_wine
for column in df_wine.columns:
    df_wine[column] = df_wine[column].where(pandas.notnull(df_wine[column]), None)


conn.rollback()
cursor = conn.cursor()

for index, row in df_wine.iterrows():
    product_id = str(uuid4())
    
    
    q_sub = """
        SELECT sub.subcategory_id
        FROM catalog.subcategory sub
        JOIN catalog.category cat ON sub.category_id = cat.category_id
        WHERE sub.subcategory_name_fr = %s AND cat.category_name_fr = %s;
    """
    cursor.execute(q_sub, (row['appellation'], row['color']))

    subcategory_id = cursor.fetchone()
    print(subcategory_id)
    product_name_fr = row['name']
    product_name_en = row['name']
    product_name_es = row['name']
    product_upc = row['code'] if row['code'] is not None else 0
    producer_code = row['code'] if row['code'] is not None else 'None'
    details = {
        key: (None if isinstance(value, float) and np.isnan(value) else value)
        for key, value in {
            'appellation': row['appellation'],
            'color': row['color'],
            'country': row['country'],
            'format': row['format'],
            'region': row['region'],
            'alcohol': row['alcohol'],
            'sugar': row['sugar'],
            'producer': row['producer'],
            'agent': row['agent'],
            'tasting': row['tasting'],
            'aging': row['aging'],
            'temperature': row['temperature'],
            'grapes': row['grapes'],
            'continent': row['continent'],
            'category': row['category'],
            'alcoholType': row['alcoholType'],
            'millesime': row['millesime']
        }.items()
    }
    details_json = json.dumps(details, default=str)  # Ensures None becomes "null"

    
    if subcategory_id is None:
        print("subcategory not found")
        continue
    q_insert = """
        INSERT INTO catalog.product (product_id, product_name_fr, product_name_en, product_name_es, product_upc, producer_code, subcategory_id, details) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(q_insert, (
        product_id, row['name'], row['name'], row['name'], 
        product_upc, producer_code, subcategory_id[0], details_json
    ))
    conn.commit()



cursor.close()  

Number of wine entries: 9287
Index(['name', 'appellation', 'format', 'country', 'region', 'price',
       'alcohol', 'sugar', 'color', 'producer', 'agent', 'code', 'upc',
       'tasting', 'aging', 'temperature', 'grapes', 'misc', 'continent',
       'category', 'alcoholType', 'millesime'],
      dtype='object')
('40536035-5d6e-4f33-94a7-578741a94cbf',)
('992057a7-f1c4-453c-ba4e-19b353dc05ca',)
('4ca28ea9-9e8f-4018-a945-19d1650b0feb',)
('c9a4d1e3-c609-4905-8cae-425a2f608031',)
('289f3490-fb4e-42d5-b6cd-dccad1166276',)
('eb07db3a-ae61-437e-a059-319b4fce95f1',)
('a7a1652c-d1cd-44a8-b6f0-531067004300',)
('c9a4d1e3-c609-4905-8cae-425a2f608031',)
('992057a7-f1c4-453c-ba4e-19b353dc05ca',)
('d34fc6ea-d873-405d-860c-964d96cf7f99',)
('9e8dc336-49c1-497e-9508-679129cb051f',)
('9e8dc336-49c1-497e-9508-679129cb051f',)
('3a6815ce-1f39-462d-b930-d888bbfb6dae',)
('c9a4d1e3-c609-4905-8cae-425a2f608031',)
('c9a4d1e3-c609-4905-8cae-425a2f608031',)
('992057a7-f1c4-453c-ba4e-19b353dc05ca',)
('992057a7-f1c

In [54]:
conn.rollback()
cursor = conn.cursor()
cursor.execute("SELECT producer_code, product_name_fr FROM catalog.product;")
row = cursor.fetchone()
code = row[0]  # row[0] is already a dictionary
print(code)
print(row)


101394
('101394', 'Reynac')


# INSERT BEERS

In [56]:
import json
import pandas
from uuid import uuid4
import numpy as np
with open("../saq_cat.json") as f:
    data = json.load(f)

# put data into a dataframe
df = pandas.DataFrame(data)
df.head()
beer_df = df[df["category"] == "CATEGORY_BEER"].copy()
print(f"Number of beer entries: {len(beer_df)}")

color_mapping = {
    'Blond': 'Blonde',
    'Blanc': 'Blanche',
    'Brun': 'Brune',
    'Doré': 'Dorée',
    'Ambré': 'Ambrée',
    'Roux, rousse': 'Rousse',
    'Noir': 'Noire',
    'Rosé': 'Rosée'
}


beer_df['color'] = beer_df['color'].fillna('None').str.replace(r'\(e\)', '', regex=True)
beer_df['color'] = beer_df['color'].replace(color_mapping)
print(beer_df['color'].unique())

print(beer_df.head())
print(beer_df.columns)

Number of beer entries: 126
['Blonde' 'Ambrée' 'Blanche' 'Brune' 'Dorée' 'Noire' 'Rouge' 'Jaune'
 'Rousse' 'Rosée']
                          name appellation  format    country region   price  \
24                Faxe Premium        None  500 ml   Danemark   None  2,50 $   
25            Faxe Extra Forte        None  500 ml   Danemark   None  3,00 $   
96          Radeberger Pilsner        None  500 ml  Allemagne   None  3,05 $   
183                 Orval 2017        None  330 ml   Belgique   None  4,45 $   
184  Schneider Weisse original        None  500 ml  Allemagne   None  3,90 $   

    alcohol sugar    color               producer  ...             upc  \
24      5 %  None   Blonde          Faxe Bryggeri  ...  05741000193006   
25     10 %  None   Blonde          Faxe Bryggeri  ...  05741000007624   
96    4,8 %  None   Blonde   Radeberger Gruppe KG  ...  04014388000213   
183   6,2 %  None   Ambrée   Brasserie d'Orval SA  ...  06784544000019   
184   5,4 %  None  Blanche  Schne

In [57]:
import json
import pandas
from uuid import uuid4
import numpy as np
with open("../saq_cat.json") as f:
    data = json.load(f)

# put data into a dataframe
df = pandas.DataFrame(data)
df.head()
beer_df = df[df["category"] == "CATEGORY_BEER"].copy()

color_mapping = {
    'Blond': 'Blonde',
    'Blanc': 'Blanche',
    'Brun': 'Brune',
    'Doré': 'Dorée',
    'Ambré': 'Ambrée',
    'Roux, rousse': 'Rousse',
    'Noir': 'Noire',
    'Rosé': 'Rosée'
}


beer_df['color'] = beer_df['color'].fillna('None').str.replace(r'\(e\)', '', regex=True)
beer_df['color'] = beer_df['color'].replace(color_mapping)


conn.rollback()
cursor = conn.cursor()
i=0
for index, row in beer_df.iterrows():

    product_id = str(uuid4())
      
    q_sub = """
        SELECT sub.subcategory_id
        FROM catalog.subcategory sub
        JOIN catalog.category cat ON sub.category_id = cat.category_id
        WHERE sub.subcategory_name_fr = %s AND cat.category_name_fr = %s;
    """
    cursor.execute(q_sub, (row['country'], row['color']))
    


    subcategory_id = cursor.fetchone()
    print(subcategory_id)
    i +=1
    product_name_fr = row['name']
    product_name_en = row['name']
    product_name_es = row['name']
    product_upc = row['upc'] if row['upc'] is not None else 0
    producer_code = row['code'] if row['code'] is not None else 'None'
    details = {
        key: (None if isinstance(value, float) and np.isnan(value) else value)
        for key, value in {
            'appellation': row['appellation'],
            'color': row['color'],
            'country': row['country'],
            'format': row['format'],
            'region': row['region'],
            'alcohol': row['alcohol'],
            'sugar': row['sugar'],
            'producer': row['producer'],
            'agent': row['agent'],
            'tasting': row['tasting'],
            'aging': row['aging'],
            'temperature': row['temperature'],
            'grapes': row['grapes'],
            'continent': row['continent'],
            'category': row['category'],
            'alcoholType': row['alcoholType'],
            'millesime': row['millesime']
        }.items()
    }
    details_json = json.dumps(details, default=str)  # Ensures None becomes "null"

    
    if subcategory_id is None:
        print("subcategory not found")
        continue
    q_insert = """
        INSERT INTO catalog.product (product_id, product_name_fr, product_name_en, product_name_es, product_upc, producer_code, subcategory_id, details) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(q_insert, (
        product_id, row['name'], row['name'], row['name'], 
        product_upc, producer_code, subcategory_id[0], details_json
    ))
    conn.commit()
cursor.close()  
print(f"Number of beer entries: {i}")

('800dadcf-b064-43f7-8d82-511da774ad16',)
('800dadcf-b064-43f7-8d82-511da774ad16',)
('c5fb5aed-3ff8-406d-9678-895e9354dade',)
('4ad22ee0-a5e3-417b-85b2-70c4018b68ec',)
('5d33295b-49ed-4737-8da3-275bdaaa7ab7',)
('c5fb5aed-3ff8-406d-9678-895e9354dade',)
('e2c1a9c5-ba1d-467a-bf9d-21226bf45ece',)
('c23fdc43-176f-49a5-91bd-e5bef49b29f3',)
('27590f32-5d93-4333-bef7-57e3736a70d0',)
('4a44ec31-c09b-497f-ab7b-5d0128c1254e',)
('be49e4e6-1bbd-464b-86ba-694ac7697fd6',)
('41b7fe36-e326-4e23-ab89-b14d88b2c856',)
('6cf3787c-d1e3-44f4-8cfc-c4b65cb77162',)
('be49e4e6-1bbd-464b-86ba-694ac7697fd6',)
('846dac79-45f4-4808-8afd-348735f2638e',)
('491c120c-e86a-4970-9c41-031349679b07',)
('c106dafd-12e9-4389-a964-c4eca2f9fd30',)
('4d1e2a32-56d4-4fb5-b30b-e31d37d150f7',)
('1d791cd7-576b-4263-b38d-348e176e32b1',)
('800dadcf-b064-43f7-8d82-511da774ad16',)
('5d33295b-49ed-4737-8da3-275bdaaa7ab7',)
('1b986810-b3d8-458e-8ad8-4094d26a880c',)
('491c120c-e86a-4970-9c41-031349679b07',)
('34c1f696-fae4-4f33-8656-f579f3c5

# GET IMAGES

In [ ]:

from zenrows import ZenRowsClient

client = ZenRowsClient("4974bf17a231d52157b7e296435f98c61fcc80e7")
url = "https://httpbin.io/anything"

response = client.get(url)

print(response.text)

In [ ]:
#create a funtion tha takes a producer code and fetches the image from SAQ product page with base_url = "https://saq.com/"+ + producer code
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

def fetch_image(producer_code):
    base_url = "https://www.saq.com/"
    url = urljoin(base_url, producer_code)
    response = requests.get(url)



In [27]:
import json
with open("../saq_cat.json") as f:
    data = json.load(f)


i=0
for item in data:
    if item['code'] is not None:
        print(item['code'])
        i+=1
    if i==5:
        break

101394
10157
1016
10204533
10209975


In [25]:
print (item.keys())

dict_keys(['name', 'appellation', 'format', 'country', 'region', 'price', 'alcohol', 'sugar', 'color', 'producer', 'agent', 'code', 'upc', 'tasting', 'aging', 'temperature', 'grapes', 'misc', 'continent', 'category', 'alcoholType'])


In [ ]:
import time
import os
import requests
from bs4 import BeautifulSoup
from zenrows import ZenRowsClient
import json

with open("../saq_cat.json") as f:
    data = json.load(f)


client = ZenRowsClient("4974bf17a231d52157b7e296435f98c61fcc80e7")

def get_saq_image(product_code, retries=3, delay=1, save_dir="../images"):
    url = f"https://www.saq.com/fr/{product_code}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    for attempt in range(retries):
        try:
            response = client.get(url, headers=headers, timeout=5)
            if response.status_code == 200:
                break  # Success, exit loop
            print(f"Attempt {attempt + 1}: Received {response.status_code}, retrying...")
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1}: Error {e}, retrying...")
        time.sleep(delay * (2 ** attempt))  # Exponential backoff
    else:
        print(f"Failed to fetch page after {retries} retries.")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    img_tag = soup.select_one('a.MagicZoom')

    if img_tag and 'href' in img_tag.attrs:
        image_url = img_tag['href']
        print(f"Image URL: {image_url}")

        # Download image using ZenRowsClient
        os.makedirs(save_dir, exist_ok=True)
        image_path = os.path.join(save_dir, f"{product_code}.jpg")

        try:
            img_response = client.get(image_url, headers=headers, timeout=5)
            if img_response.status_code == 200:
                with open(image_path, "wb") as f:
                    f.write(img_response.content)
                print(f"Image saved to {image_path}")
                return image_path
            else:
                print(f"Failed to download image, status code: {img_response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading image: {e}")

    print(f"Image not found for product {product_code}")
    return None

# Example usage



with open("../saq_cat.json") as f:
    data = json.load(f)



for item in data:
    if item['code'] is not None:
        get_saq_image(item['code'])
        






Image URL: https://www.saq.com/media/catalog/product/1/0/101394-1_1673300460.png?optimize=high&fit=bounds&height=&width=&format=jpeg
Image saved to ../images/101394.jpg


In [67]:
import time
import psycopg2

def update_product_images(conn, batch_size=10):
    conn.rollback()  # Reset any previous transaction state

    with conn.cursor() as cursor:
        cursor.execute("SELECT product_id, producer_code FROM catalog.product;")
        products = cursor.fetchall()  # Fetch all rows at once

    updated_count = 0

    with conn.cursor() as cursor:
        for product_id, producer_code in products:
            try:
                image_url = get_saq_image(producer_code)  # Use the correct function
                if image_url:
                    cursor.execute(
                        "UPDATE catalog.product SET product_img_url = %s WHERE product_id = %s;",
                        (image_url, product_id),
                    )
                    updated_count += 1

                # Commit in batches for better performance
                if updated_count % batch_size == 0:
                    conn.commit()
                    print(f"Committed {updated_count} updates...")

                time.sleep(1)  # Be nice to SAQ servers

            except Exception as e:
                print(f"Error updating product {product_id}: {e}")
                conn.rollback()  # Prevent locking issues

        conn.commit()  # Final commit at the end
        print(f"Total updated: {updated_count}")


update_product_images(conn)


Image URL: https://www.saq.com/media/catalog/product/1/0/10220373-1_1636388167.png?optimize=high&fit=bounds&height=&width=&format=jpeg
Image URL: https://www.saq.com/media/catalog/product/1/0/102210-1_1580592010.png?optimize=high&fit=bounds&height=&width=&format=jpeg
Attempt 1: Received 404, retrying...
Attempt 2: Received 404, retrying...
Attempt 3: Received 404, retrying...


KeyboardInterrupt: 

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT * from catalog.measure_lkp ")
row = cursor.fetchall()
print(row)




[('a9e5e8dd-098c-4f76-86f8-08f36620ac0c', '4mwgSAFKoNSRfDIZykoL', 'MEASURE', 'unité', 'unité', 'unité', 'un', 'un', 'un', None, None, 1.0), ('4e6ac9c7-cba5-41ea-ba56-40b27c7a455d', '3yEPH8i3xLX8vwwLjfkd', 'MEASURE', 'gramme', 'gramme', 'gramme', 'g', 'g', 'g', None, None, 1.0), ('90b12e67-c46c-455c-82d6-f3187df715cd', 'RtSe4oh5jnIeUSdNH74A', 'MEASURE', 'livre', 'livre', 'livre', 'lb', 'lb', 'lb', '3yEPH8i3xLX8vwwLjfkd', None, 453.592), ('db7e83f9-dc52-41a1-89b8-0d51f9d09b6b', 'A1qRtAMFq2xsgKbBSm9Y', 'MEASURE', 'once', 'once', 'once', 'oz', 'oz', 'oz', '3yEPH8i3xLX8vwwLjfkd', None, 28.3495), ('3e714f8c-e0a5-4b24-b128-dd6f9e584538', '0NZwvGn10sWlTvPDFaOA', 'MEASURE', 'kilogramme', 'kilogramme', 'kilogramme', 'kg', 'kg', 'kg', '3yEPH8i3xLX8vwwLjfkd', None, 1000.0), ('67336901-14a1-4162-8dfd-8a5ea67e3b15', 'CAVv1ztEr5iTdBHwAXSk', 'MEASURE', 'milligramme', 'milligramme', 'milligramme', 'mg', 'mg', 'mg', '3yEPH8i3xLX8vwwLjfkd', None, 0.001), ('0dfb50b4-6e0d-4d3c-8167-302bed88d06d', '4BPmmhD4

In [40]:
import json
with open("../saq_cat.json") as f:
    data = json.load(f)
print(type(data))

<class 'list'>


In [60]:
import json
import re
def extract_data(products):
    extracted = []
    for product in products:
        format_match = re.match(r"(\d+)\s*(ml|L)", product.get("format", ""), re.IGNORECASE)
        quantity, unit = (int(format_match.group(1)), format_match.group(2)) if format_match else (None, None)
        
        extracted.append({
            "code": product.get("code"),
            "upc": product.get("upc"),
            "quantity": quantity,
            "unit": unit
        })
    return extracted

extracted_data = extract_data(data)
print(extracted_data[0:10])

[{'code': '101394', 'upc': '03250510301105', 'quantity': 750, 'unit': 'ml'}, {'code': '10157', 'upc': '00080432500170', 'quantity': 750, 'unit': 'ml'}, {'code': '1016', 'upc': '08001333700027', 'quantity': 1, 'unit': 'L'}, {'code': '10204533', 'upc': '03760090790430', 'quantity': 750, 'unit': 'ml'}, {'code': '10209975', 'upc': '03586153000700', 'quantity': 700, 'unit': 'ml'}, {'code': '10210651', 'upc': '00085592160158', 'quantity': 750, 'unit': 'ml'}, {'code': '10210677', 'upc': '05281022512442', 'quantity': 750, 'unit': 'ml'}, {'code': '10211397', 'upc': '03760027690130', 'quantity': 750, 'unit': 'ml'}, {'code': '10213860', 'upc': '00089540462875', 'quantity': 50, 'unit': 'ml'}, {'code': '10218783', 'upc': '03760090820342', 'quantity': 750, 'unit': 'ml'}]


## Insert Quantities

In [58]:
import psycopg2

# Connect to the default "postgres" database to create a new one
conn = psycopg2.connect(
    host="34.47.32.127",
    port=5432,  # Default PostgreSQL port
    dbname="octogone_cat_v0",
    user="postgres",
    password="f87JpR9Uvud6NR3HwbP"
)


In [61]:
# insert units
unit_equivalence_id = 'a9e5e8dd-098c-4f76-86f8-08f36620ac0c'
volume_equivalence_measure_id_ml = '0dfb50b4-6e0d-4d3c-8167-302bed88d06d'
volume_equivalence_measure_id_l = '81f132d1-de62-4584-9c5e-6ab5f2b3ae21'
conn.rollback()
for item in extracted_data:
    if item['unit'] is not None:
        if item['unit'].lower() == 'ml':
            unit_id = volume_equivalence_measure_id_ml
        elif item['unit'].lower() == 'l':
            unit_id = volume_equivalence_measure_id_l
        else:
            unit_id = None
        cursor = conn.cursor()
        cursor.execute(""" UPDATE catalog.product SET product_upc = %s , unit_equivalence_id = %s, volume_equivalence_measure_id = %s, unit_equivalence_qty = 1,
                       volume_equivalence_qty =%s WHERE producer_code = %s""", (item['upc'], unit_equivalence_id, unit_id, item['quantity'], item['code']))
        conn.commit()




In [14]:
conn.rollback()

# SCRAPE product Images


In [52]:
import psycopg2

# Connect to the default "postgres" database to create a new one
conn = psycopg2.connect(
    host="34.47.32.127",
    port=5432,  # Default PostgreSQL port
    dbname="octogone_cat_v0",
    user="postgres",
    password="f87JpR9Uvud6NR3HwbP"
)



cursor = conn.cursor()
cursor.execute("SELECT * from catalog.subcategory")
print(cursor.fetchmany(5))


[('4ddbfdd3-e3eb-4e03-b9af-fe82d3efff77', '89f90469-a407-40ed-bc72-4591e69573d7', 10000604, None, 'Fruit/Nuts/Seeds Combination Variety Packs', None, 'FRUIT NUTS SEEDS COMBINATION VARIETY PACKS', None), ('dc6d88e4-66d2-4d41-b5a3-467df20e719b', '5666dee7-19f2-42ea-9c05-eed63ff74297', 10000605, None, 'Fruits/Vegetables/Nuts/Seeds Variety Packs', None, 'FRUITS VEGETABLES NUTS SEEDS VARIETY PACKS', None), ('42451770-2153-497e-a16f-7edbd9665d3d', 'c90bd094-d726-4a41-ab1b-26cd97e63b2f', 30002162, None, 'SEA URCHIN', None, 'AQUATIC INVERTEBRATES PREPARED PROCESSED FROZEN SEA URCHIN', None), ('01c77e2e-99f4-4c51-8ef0-f775586a5ef1', 'c90bd094-d726-4a41-ab1b-26cd97e63b2f', 30001715, None, 'OCTOPUS', None, 'AQUATIC INVERTEBRATES PREPARED PROCESSED FROZEN OCTOPUS', None), ('3b7d3a57-0020-4c1e-bdef-715def6fa645', 'c90bd094-d726-4a41-ab1b-26cd97e63b2f', 30000179, None, 'ANEMONES', None, 'AQUATIC INVERTEBRATES PREPARED PROCESSED FROZEN ANEMONES', None)]


                                      0                                     1  \
0  4ddbfdd3-e3eb-4e03-b9af-fe82d3efff77  89f90469-a407-40ed-bc72-4591e69573d7   
1  dc6d88e4-66d2-4d41-b5a3-467df20e719b  5666dee7-19f2-42ea-9c05-eed63ff74297   
2  42451770-2153-497e-a16f-7edbd9665d3d  c90bd094-d726-4a41-ab1b-26cd97e63b2f   
3  01c77e2e-99f4-4c51-8ef0-f775586a5ef1  c90bd094-d726-4a41-ab1b-26cd97e63b2f   
4  3b7d3a57-0020-4c1e-bdef-715def6fa645  c90bd094-d726-4a41-ab1b-26cd97e63b2f   

          2     3                                           4     5  \
0  10000604  None  Fruit/Nuts/Seeds Combination Variety Packs  None   
1  10000605  None  Fruits/Vegetables/Nuts/Seeds Variety Packs  None   
2  30002162  None                                  SEA URCHIN  None   
3  30001715  None                                     OCTOPUS  None   
4  30000179  None                                    ANEMONES  None   

                                                   6     7  
0         FRUIT NUTS SEED

AttributeError: 'numpy.int64' object has no attribute 'sort_values'